## 4.1.1 build model from block

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

In [3]:
X = tf.random.uniform((2,20))
net = MLP()
net(X)

<tf.Tensor: id=62, shape=(2, 10), dtype=float32, numpy=
array([[ 0.25095996, -0.24893613, -0.45473456, -0.04311958, -0.2295383 ,
         0.27443463,  0.37376237, -0.14436285, -0.37055504, -0.00098359],
       [ 0.1769592 , -0.14420828, -0.3330693 , -0.14657606, -0.14307055,
         0.27779722,  0.33585754, -0.14960271, -0.38496172,  0.10907309]],
      dtype=float32)>

## 4.1.2 Sequential

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(X)

<tf.Tensor: id=117, shape=(2, 10), dtype=float32, numpy=
array([[ 0.18656988, -0.24001493, -0.22437742, -0.15284519, -0.10276742,
        -0.04557859,  0.49852332,  0.04370471, -0.26982784, -0.14277868],
       [-0.11792175, -0.28876868, -0.4724299 , -0.08530779, -0.17974694,
         0.07656129,  0.53052086,  0.13260414, -0.36211073, -0.08055006]],
      dtype=float32)>

## 4.1.3 build complex model

In [5]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

In [6]:
net = FancyMLP()
net(X)

<tf.Tensor: id=229, shape=(), dtype=float32, numpy=28.175375>

## debugging

In [7]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation=tf.nn.relu),
            tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)

    
    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

TypeError: add() takes 2 positional arguments but 4 were given